In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from mecab import MeCab

mecab = MeCab()

def get_top_n_structure_similar(df, target_sentence, n=15):
    # "msg" 컬럼(문장 리스트) → 품사 시퀀스 변환
    
    err_sentence_list = df["err_sentence"].tolist()
    sentence_list = df["cor_sentence"].tolist()
    sentence_list_pos = [" ".join([tag for _, tag in mecab.pos(s)]) for s in sentence_list]
    target_pos = " ".join([tag for _, tag in mecab.pos(target_sentence)])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([target_pos] + sentence_list_pos)
    sims = cosine_similarity(X[0], X[1:])[0]
    
    top_idxs = sims.argsort()[::-1][:n]
    # 결과를 (원문, 유사도) 튜플로 반환
    #return [err_sentence_list[idx] for idx in top_idxs], [(sentence_list[idx], sims[idx]) for idx in top_idxs]
    return list(zip([err_sentence_list[idx] for idx in top_idxs], [sentence_list[idx] for idx in top_idxs]))
    

In [2]:
df =pd.read_csv('data/train_dataset.csv')

print(get_top_n_structure_similar(df, "칭찬을 받으니 부끄럽자 얼굴이 빨개졌다."))


[('칭찬을 받으니 부끄럽자 얼굴이 빨개졌다.', '칭찬을 받으니 부끄러워서 얼굴이 빨개졌다.'), ('근육이 뭉쳐서 토도한의원에 가서 부황을 떴다.', '근육이 뭉쳐서 토도한의원에 가서 부항을 떴다.'), ('점심을 너무 많이 먹어 배가 부르자 잠이 쏟아졌다.', '점심을 너무 많이 먹어 배가 불러서 잠이 쏟아졌다.'), ('벼락소리가 너무 크고 무섭자 아기가 울음을 터뜨렸다.', '벼락소리가 너무 크고 무서워서 아기가 울음을 터뜨렸다.'), ('강아지가 가려운지 발로 귀를 부볐다.', '강아지가 가려운지 발로 귀를 비볐다.'), ('신호등이 빨강색으로 바뀌자 차들이 일제히 멈춰 섰다.', '신호등이 빨간색으로 바뀌자 차들이 일제히 멈춰 섰다.'), ('화장실 누수가 발생해서 옆집에 큰 피해를 주게 되었다.', '화장실 누수가 발생해서 옆집에 큰 피해를 주게 되었다.'), ('그는 머리기름을 바르고 빗으로 머리를 단정하게 빗어 넘겼다.', '그는 머릿기름을 바르고 빗으로 머리를 단정하게 빗어 넘겼다.'), ('영화가 너무 슬프자 눈물이 멈추지 않았다.', '영화가 너무 슬퍼서 눈물이 멈추지 않았다.'), ('저녁 식사가 끝나자마자 산더미처럼 쌓인 설겆이거리를 보고 한숨부터 나왔다. 오늘은 내가 당번이니 어쩔 수 없이 팔을 걷어붙였다.', '저녁 식사가 끝나자마자 산더미처럼 쌓인 설거지거리를 보고 한숨부터 나왔다. 오늘은 내가 당번이니 어쩔 수 없이 팔을 걷어붙였다.'), ('박 과장이 조용히 다가와 내일 있을 인사이동에 대해 살짝 귀뜸해주었다. 덕분에 미리 마음의 준비를 할 수 있게 되어 고마웠다.', '박 과장이 조용히 다가와 내일 있을 인사이동에 대해 살짝 귀띔해주었다. 덕분에 미리 마음의 준비를 할 수 있게 되어 고마웠다.'), ('박영식의 터무니없는 변명을 듣고 있자니 정말 어의가 없었다. 처음부터 끝까지 앞뒤가 맞지 않는 말들로 상황을 모면하려는 모습이 뻔히 보였기 때문이다. 결국, 더는 대화를 이어 나갈 의미가 없다고 판단하고 자리를 떠났다.', 

In [3]:
import os
import argparse

import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI
from prompts import baseline_prompt
import promptTemplate
import re
import importlib


importlib.reload(promptTemplate)

def extract_answer(text):
    # [Answer]: 뒤에 오는 줄의 맨 앞~줄 끝까지 추출
    match = re.search(r"\[Answer\]:\s*(.*)", text)
    if match:
        return match.group(1).strip()
    return ""


def main():

    args = {"input":"data/train_dataset.csv","model":"solar-pro2","output":"eval_submission.csv"}
    

    # Load environment variables
    load_dotenv()
    
    # Load data
    df = pd.read_csv(args['input'])
    
    if "err_sentence" not in df.columns:
        raise ValueError("Input CSV must contain 'err_sentence' column")

    # Setup Upstage client
    api_key = os.getenv("UPSTAGE_API_KEY")
    if not api_key:
        raise ValueError("UPSTAGE_API_KEY not found in environment variables")
    
    print(f"Model: {args['model']}")
    print(f"Output: {args['output']}")

    err_sentences = []
    cor_sentences = []
    
    # Process each sentence
    for row in tqdm(df.itertuples(), total=len(df), desc="Generating"):

        text = row.err_sentence
        
        err_sentences.append(text)

        examples = get_top_n_structure_similar(df, text)
        
        try:
            resp = promptTemplate.process(text, examples[1:])
            corrected = extract_answer(resp)
            cor_sentences.append(corrected)
            
        except Exception as e:
            print(f"Error processing: {text[:50]}... - {e}")
            cor_sentences.append(text)  # fallback to original

    # Save results with required column names
    out_df = pd.DataFrame({"err_sentence": err_sentences, "cor_sentence": cor_sentences})
    out_df.to_csv(args['output'], index=False)
    print(f"Wrote {len(out_df)} rows to {args['output']}")


In [4]:
import time
import subprocess
import pandas as pd

if __name__ == "__main__":

   
    main()

    time.sleep(3)

    # 명령어 실행 및 콘솔 출력
    cmd = "uv run python evaluate.py --true_df data/train_dataset.csv --pred_df eval_submission.csv --output analysis.csv"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print("콘솔 출력:")
    print(result.stdout)

    # 파일 결과 확인
    df = pd.read_csv("analysis.csv")
    print("analysis.csv 결과:")
    print(df)
    


Model: solar-pro2
Output: eval_submission.csv


Generating: 100%|██████████| 254/254 [31:33<00:00,  7.45s/it] 


Wrote 254 rows to eval_submission.csv
콘솔 출력:
=== 평가 결과 ===
Recall: 41.63%
Precision: 25.76%

Analysis results saved to analysis.csv

analysis.csv 결과:
                                              original  \
0    이날 재판에선 박 전 대통령이 피고인석에 앉은 모습은 처음으로 언론에 공개됐다. 재...   
1    하늘에 먹구름이 몰려오더니 금새 비원 앞 광장에 비가 쏟아지기 시작했다. 기상청 아...   
2    불상사를 예방하려면 중국 방문시 발급 받은 비자가 허가하는 체류 기간을 확인하고 입...   
3                               -재팬 쇼크는 유럽 경제에 끼친 파장은.   
4    무대 위엔 으스스한 분위기의 문 하나, 작은 의자와 배우 한 명뿐. 처음 자신을 브...   
..                                                 ...   
249  귀여운 디자인으로 눈길을 사로잡는 기아차 레이 일렉트릭은 롱레인지 모델인 '시그니처...   
250                               몸이 피곤하자 바로 잠자리에 들었다.   
251                   폭우 북상에 따라 정부는 긴급 비대책 회의를 소집했습니다.   
252  야구, 축구처럼 상대적으로 체계가 잘 갖춰진 종목도 애 먹긴 마찬가지다. 한 대학 ...   
253                  해수청은 최근 경제 동향에 대한 분석 보고서를 발표했습니다.   

                                                golden  \
0    이날 재판에선 박 전 대통령이 피고인석에 앉은 모습이 처음으로 언론에 공개됐다. 재...   
1    하늘에 먹구름이 몰려오더니 금세 비원 앞 광장에 비가 쏟아